In [1]:
! pip install --upgrade pip
!pip install --upgrade lifelines
!pip install plotly seaborn
!pip install xgboost 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 29.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=308d8ce1d8a2767219f8ae2e68e86e8462ccf6ef7d7e9811cc1e71ac4db62e56
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer
from lifelines.utils import concordance_index

In [3]:
# Load the dataset
train_data = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test_data = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
data_dict = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/data_dictionary.csv')

In [13]:
train_data.head()

,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,...,tce_div_match_GvH non-permissive,tce_div_match_HvG non-permissive,tce_div_match_Permissive mismatched,donor_related_Related,donor_related_Unrelated,"melphalan_dose_N/A, Mel not given",cardiac_Not done,cardiac_Yes,pulm_moderate_Not done,pulm_moderate_Yes
0,0,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,False,False,False,False,True,True,False,False,False,False
1,1,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,False,False,True,True,False,True,False,False,False,True
2,2,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,False,False,True,True,False,True,False,False,False,False
3,3,2.0,8.0,6.0,6.0,10.0,2.0,6.0,2.0,2.0,...,False,False,True,False,True,True,False,False,False,False
4,4,2.0,8.0,6.0,6.0,10.0,2.0,5.0,2.0,2.0,...,False,False,True,True,False,False,False,False,False,False


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28800 entries, 0 to 28799
Columns: 151 entries, ID to pulm_moderate_Yes
dtypes: bool(126), float64(23), int64(2)
memory usage: 9.0 MB


In [15]:
train_data.describe()

,ID,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,...,age_at_hct,hla_match_a_low,hla_match_b_high,comorbidity_score,karnofsky_score,hla_low_res_8,hla_match_drb1_high,hla_low_res_10,efs,efs_time
count,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,...,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000,28800.000000
mean,14399.500000,1.802292,7.104132,5.240590,5.272639,8.961146,1.784375,5.282708,1.781354,1.741424,...,38.663162,1.733229,1.742257,1.690694,84.018403,7.042535,1.741215,8.899479,0.539306,23.237678
std,8313.988213,0.405107,1.468144,1.169154,1.150039,1.756146,0.417713,1.150898,0.419919,0.437860,...,21.147581,0.446110,0.443470,1.979887,10.912164,1.507241,0.443568,1.783208,0.498461,24.799748
min,0.000000,0.000000,2.000000,2.000000,0.000000,3.000000,0.000000,2.000000,0.000000,1.000000,...,0.044000,0.000000,0.000000,0.000000,40.000000,2.000000,0.000000,4.000000,0.000000,0.333000
25%,7199.750000,2.000000,7.000000,5.000000,5.000000,9.000000,2.000000,5.000000,2.000000,1.000000,...,19.539000,1.000000,1.000000,0.000000,70.000000,6.000000,1.000000,8.000000,0.000000,5.619750
50%,14399.500000,2.000000,8.000000,6.000000,6.000000,10.000000,2.000000,6.000000,2.000000,2.000000,...,41.006000,2.000000,2.000000,1.000000,90.000000,8.000000,2.000000,10.000000,1.000000,9.796500
75%,21599.250000,2.000000,8.000000,6.000000,6.000000,10.000000,2.000000,6.000000,2.000000,2.000000,...,55.965250,2.000000,2.000000,2.000000,90.000000,8.000000,2.000000,10.000000,1.000000,35.100000
max,28799.000000,2.000000,8.000000,6.000000,6.000000,10.000000,2.000000,6.000000,2.000000,2.000000,...,73.726000,2.000000,2.000000,10.000000,100.000000,8.000000,2.000000,10.000000,1.000000,156.819000


In [4]:
# Encode categorical variables
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Align train and test data columns
train_data, test_data = train_data.align(test_data, join='left', axis=1, fill_value=0)


In [5]:
# Fill missing values for numeric columns with median
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())

# Fill missing values for categorical columns with the mode
categorical_cols = train_data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    train_data[col].fillna(train_data[col].mode()[0], inplace=True)

In [6]:
# Repeat for the test dataset
test_data[numeric_cols] = test_data[numeric_cols].fillna(test_data[numeric_cols].median())
for col in categorical_cols:
    test_data[col].fillna(test_data[col].mode()[0], inplace=True)

# Prepare features and target for training
X_train = train_data.drop(columns=['efs', 'efs_time'])
y_train = train_data['efs']

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [9]:
# Split the data for validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Initialize models for boosting
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    
}

# Function to evaluate models
def evaluate_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)
    c_index = concordance_index(y_val, y_val_pred)
    print(f'Model: {model.__class__.__name__}, Validation Concordance Index: {c_index:.4f}')
    return model

In [10]:

# Evaluate each boosting model and store the best
best_model = None
best_c_index = 0
for model_name, model in models.items():
    trained_model = evaluate_model(model, X_train_split, y_train_split, X_val, y_val)
    # Update best model if necessary
    if best_model is None or concordance_index(y_val, trained_model.predict(X_val)) > best_c_index:
        best_model = trained_model
        best_c_index = concordance_index(y_val, trained_model.predict(X_val))

Model: RandomForestRegressor, Validation Concordance Index: 0.7138
Model: AdaBoostRegressor, Validation Concordance Index: 0.6593
Model: GradientBoostingRegressor, Validation Concordance Index: 0.7418


In [11]:
# Prepare features and target for training
X_train = train_data.drop(columns=['efs', 'efs_time'])
y_train = train_data['efs']

# Handle missing values for numeric columns
numeric_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
X_train[numeric_cols] = X_train[numeric_cols].fillna(X_train[numeric_cols].median())

# Handle missing values for categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
for col in categorical_cols:
    X_train[col].fillna(X_train[col].mode()[0], inplace=True)

# Encode categorical variables
X_train = pd.get_dummies(X_train, drop_first=True)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Prepare the test set in the same way
X_test = test_data

# Handle missing values for numeric columns in the test set
X_test[numeric_cols] = X_test[numeric_cols].fillna(X_test[numeric_cols].median())
for col in categorical_cols:
    X_test[col].fillna(X_test[col].mode()[0], inplace=True)

# Encode categorical variables in the test set
X_test = pd.get_dummies(X_test, drop_first=True)

# Align train and test data columns
X_train_scaled, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Scale the test data
X_test_scaled = scaler.transform(X_test)

# Make predictions on the test set
test_predictions = model.predict(X_test_scaled)

# Create submission DataFrame
submission = pd.DataFrame({'ID': test_data['ID'], 'prediction': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [12]:
# Make predictions on the test set
test_predictions = best_model.predict(X_test_scaled)

# Create submission DataFrame
submission = pd.DataFrame({'ID': test_data['ID'], 'prediction': test_predictions})

# Save to CSV
submission.to_csv('submission.csv', index=False)

# Display the first few rows of the submission
submission.head()

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


,ID,prediction
0,28800,0.336928
1,28801,0.472287
2,28802,0.259337
